In [31]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sp
from scipy.integrate import odeint

In [39]:
#Initializations

x0 = [10,0,np.pi/2,-60,0,-np.pi/2]
u = [2, -np.pi/18, 12, np.pi/25]
L = 0.5
num_time_steps = 400
delta_t = 0.1
time = np.linspace(0, num_time_steps*delta_t, num_time_steps)


In [40]:
#Generate Nominal trajectory with no noise

def model(x, t, u, L):
    e_g = x[0]
    n_g = x[1]
    theta_g = x[2]
    e_a = x[3]
    n_a = x[4]
    theta_a = x[5]
    v_g = u[0]
    phi_g = u[1]
    v_a = u[2]
    w_a = u[3] 
    dxdt = [0]*6
    dxdt[0] = v_g*np.cos(theta_g)
    dxdt[1] = v_g*np.sin(theta_g)
    dxdt[2] = (v_g/L)*np.tan(phi_g)
    dxdt[3] = v_a*np.cos(theta_a)
    dxdt[4] = v_a*np.sin(theta_a)
    dxdt[0] = w_a
    return dxdt


nominal_trajectory = odeint(model, x0, time, args = (u,L))
# xt = output[:,0]
# vt = output[:,1]

In [64]:
def get_CT_Jacobians(x,u):
    
    A = np.array([
            [0, 0, -u[0]*np.sin(x[2]), 0, 0, 0],
            [0, 0, u[0]*np.cos(x[2]), 0, 0, 0],
            [0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, -u[2]*np.sin(x[5])],
            [0, 0, 0, 0, 0, u[2]*np.cos(x[5])],
            [0, 0, 0, 0, 0, 0]])

    
    B = np.array([
            [np.cos(x[2]), 0, 0, 0],
            [np.sin(x[2]), 0, 0, 0],
            [np.tan(u[1])/L, u[0]/(L*(np.cos(u[1])**2)), 0, 0],
            [0, 0, np.cos(x[5]), 0],
            [0, 0, np.sin(x[5]), 0],
            [0, 0, 0, 1]
            ])    
    
    Ga = np.eye(6)
    
    denominator_1 = 1 + ((x[4]-x[1])/(x[3]-x[0]))**2
    numerator_11 = (x[4]-x[1]) / ((x[3] - x[0])**2)
    C11 = numerator_11/denominator_1
    
    numerator_12 = -1/(x[3]-x[0])
    C12 = numerator_12/denominator_1
    
    C14 = -1*C11
    C15 = -1*C12
    
    denominator_2 = ( (x[0]-x[3])**2 + (x[1]-x[4])**2 ) ** 0.5
    C21 = (x[0]-x[3])/denominator_2
    C22 = (x[1]-x[4])/denominator_2
    C24 = -1*C21
    C25 = -1*C22
    
    C = np.array([
            [C11, C12, -1, C14, C15, 0],
            [C21, C22, 0, C24, C25, 0],
            [C11, C12, 0, C14, C15, -1],
            [0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0]
            ])

    D = 0
    
    return A,B, Ga, C, D

def get_DT_Jacobians(x,u,dt):
    
    A,B,Gamma,C,D = get_CT_Jacobians(x,u)
    
    F = np.eye(6) + dt*A
    G = dt*B
    Omega = delta_t*Gamma
    H = C
    M = D
    
    return F,G,Omega,H,M


In [63]:
F,G,Omega,H,M = get_DT_Jacobians(x0,u,delta_t)
F

array([[ 1.00000000e+00,  0.00000000e+00, -2.00000000e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  1.22464680e-17,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.20000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  7.34788079e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [ ]:
# mu_0_A = np.array(
#     [[0],
#      [85*np.cos(np.pi/4)],
#      [0],
#      [-85*np.sin(np.pi/4)]])

# P_0_A = np.array(
#     [[10.0,0.0,0.0,0.0],
#      [0.0,2.0,0.0,0.0],
#      [0.0,0.0,10.0,0.0],
#      [0.0,0.0,0.0,2.0]
#     ])
# P_0_A = 900*P_0_A

def kf_time_update(delta_xk,delta_uk,Pk,F,G,Omega,H,R,Q):
    delta_xk_propogated = np.dot(F,delta_xk) + np.dot(G,delta_uk)
    Pk_propogated = np.dot(F,np.dot(Pk,F.transpose())) + np.dot(Omega,np.dot(Q,Omega.transpose()))    
    fc_kalman_gain = np.dot(Pk_propogated,H.transpose())
    sc_kalman_gain = np.dot(H,np.dot(Pk_propogated,H.transpose())) + R
    kalman_gain = np.dot(fc_kalman_gain,inv(sc_kalman_gain))
    return xk_propogated, Pk_propogated, kalman_gain

def kf_measurement_update(xk_propogated,yk_observed,Pk_propogated,kalman_gain,H):
    surprise_factor = yk_observed - np.dot(H,xk_propogated)
    xk_plus1 = xk_propogated + np.dot(kalman_gain,surprise_factor)
    (n,m) = Pk_propogated.shape
    I = np.identity(n)
    fc_Pk_plus1 = I - np.dot(kalman_gain,H)
    Pk_plus1 = np.dot(fc_Pk_plus1, Pk_propogated)
    return xk_plus1,Pk_plus1